In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset
import matplotlib.pyplot as plt
%matplotlib inline


torch.manual_seed(12046)

In [ ]:
# 一些超参数
context_length = 10
learning_rate = 0.01
eval_iters = 10
batch_size=1000
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
raw_datasets = load_dataset('Nan-Do/code-search-net-python')
datasets = raw_datasets['train'].filter(lambda x: 'apache/spark' in x['repo'])
# 通过索引提取datasets数据的时候，返回一个dict，其中的value是一个字符串
print(datasets[8]['original_string'])
# 当传入的是一个数组时，返回的依然是一个dict，但其中的value是一个列表
print(datasets[8: 10]['original_string'])

In [ ]:
# 手动实现tokenizer
class char_tokenizer:

  def __init__(self, data, begin_ind=0, end_ind=1):
        # 数据中出现的所有字符构成字典
        chars = sorted(list(set(''.join(data))))
        # 预留两个位置给开头和结尾的特殊字符
        self.char2ind = {s : i + 2 for i, s in enumerate(chars)}
        self.char2ind['<|b|>'] = begin_ind
        self.char2ind['<|e|>'] = end_ind
        self.begin_ind = begin_ind
        self.end_ind = end_ind
        self.ind2char = {i : s for s, i in self.char2ind.items()}
  # 实现tokenizer的编码器功能
  def encode(self, text):
        '''
        编码
        参数
        ----
        text ：str，文本
        '''
        return [self.char2ind[c] for c in text]

  # 实现tokenizer的解码器功能
  def decode(self, enc):
        '''
        解码
        参数
        ----
        enc ：int or list[int]
        '''
        if isinstance(enc, int):
            return self.ind2char[enc]
        return [self.ind2char[i] for i in enc]


In [ ]:
# 举例验证分词器
tok = char_tokenizer(datasets['original_string'])
example_text = 'def postappend(self):'
''.join(tok.decode(tok.encode(example_text))), len(tok.char2ind)

In [ ]:
# 自回归转换，将文本转换成一系列的训练数据
def autoregressive_trans(text, tokenizer, context_length=context_length):
    '''
    将文本转换成一系列的训练数据
    参数
    ----
    text ：str，文本
    tokenizer ：分词器
    context_length ：int，背景文本的长度
    返回
    ----
    inputs ：list[list[int]]，背景文本（特征）
    labels ：list[list[int]]，预测标签
    '''
    inputs, labels = [], []
    b_ind = tokenizer.begin_ind
    e_ind = tokenizer.end_ind
    enc = tokenizer.encode(text)
    # 增加开始和结尾的特殊字符
    x = [b_ind] * context_length + enc + [e_ind]
    for i in range(len(x) - context_length):
        inputs.append(x[i: i + context_length])
        labels.append(x[i + context_length])
    return inputs, labels

In [ ]:
# 举例展示自回归模式的训练数据
inputs, labels = autoregressive_trans(example_text, tok)
for a, b in zip(inputs, labels):
  print(''.join(tok.decode(a)), '------>',  tok.decode(b))

In [ ]:
# 对数据集中的所有数据进行处理
def process(data):
    '''
    在datasets的map里使用，将文本转换成训练数据
    '''
    text = data['original_string']
    # 如果是普通的map操作，传入的值是字符串
    if isinstance(text, str):
        inputs, labels = autoregressive_trans(text, tok)
        return {'inputs': inputs, 'labels': labels}
    # 如果是map操作里面batched=True，传入的值是字符串列表
    inputs, labels = [], []
    for i in text:
        i, l = autoregressive_trans(i, tok)
        inputs += i
        labels += l
    return {'inputs': inputs, 'labels': labels}

In [ ]:
# 测试process的功能
process(datasets[8:9])

In [ ]:
# 将数据分为训练集和测试集
tokenized = datasets.train_test_split(test_size=0.1, shuffle=True, seed=1024)
# 将文本转换为训练数据，里面包含inputs和labels
tokenized = tokenized.map(process, batched=True, remove_columns=datasets.column_names)
tokenized.set_format(type='torch', device=device)


In [ ]:
# 构建数据加载器
train_loader = DataLoader(tokenized['train'], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(tokenized['test'], batch_size=batch_size, shuffle=True)
# 获取一个批量的数据
next(iter(test_loader))


In [ ]:
# 定义模型
class CharMLP(nn.Module):
  def __init__(self, vs):
    super().__init__()
    '''
    根据文本背景预测下一个字母是什么
    参数
    ----
    vs ：int，字典大小
    '''
    # 文字嵌入层
    self.embedding= nn.Embedding(vs, 30)
    self.hidden1 = nn.Linear(300, 200)
    self.hidden2 = nn.Linear(200, 100)
    self.out = nn.Linear(100, vs)

  def forward(self, x):
    '''
    向前传播
    参数
    ----
    x ：torch.LongTensor，背景文本，其中的元素表示相应位置的字母在字典中的位置
    返回
    ----
    h ：torch.FloatTensor，预测结果的logits
    '''
    # 因为背景文本的长度（context_length）等于10，
    # 所以x的形状是(B, 10)，B表示批量数据的大小
    B = x.shape[0]           # (B,  10)
    emb = self.embedding(x)      # (B,  10, 30)
    h = emb.view(B, -1)       # (B, 300)
    h = F.relu(self.hidden1(h))    # (B, 200)
    h = F.relu(self.hidden2(h))    # (B, 100)
    h = self.out(h)          # (B,  vs)
    return h



In [ ]:
model = CharMLP(len(tok.char2ind)).to(device)

In [ ]:
from math import log
@torch.no_grad()
def generate(model, context, max_new_token=300):
  '''
  利用模型生成文本（反复使用模型进行预测）
  参数
  ----
  model ：CharMLP，生成文本的模型
  context ：torch.LongTensor，背景文本，形状为(1, 10)
  max_new_tokens ：int，生成文本的最大长度
  返回
  ----
  out ：list[int]，生成的文本
  '''
  out = []
  # 将模型切换到评估模式
  model.eval()
  for _ in range(max_new_token):
    logits = model(context)
    probs = F.softmax(logits, dim=-1)
    # 根据模型预测的概率，得到最终的预测结果（下一个字母）
    # 这一步运算有一定随机性
    ix = torch.multinomial(probs, num_samples=1)
    # 利用模型的预测结果更新文本背景
    context = torch.cat((context[:, 1:], ix), dim=1)
    out.append(ix.item())
    if ix.item() == tok.end_ind:
      break
  # 将模型切换至训练模式
  model.train()
  return out

In [ ]:
# 使用模型来生成文本
context = torch.zeros((1, 10), dtype=torch.long, device=device)
print(''.join(tok.decode(generate(model, context))))

In [ ]:
# 评估模型
@torch.no_grad()
def _loss(model, data_loader):
  loss = []
  data_iter = iter(data_loader)

  # 随机使用多个批量数据来预估模型效果
  for k in range(eval_iters):
    data = next(data_iter, None)
    if data is None:
      data_iter = iter(data_loader)   # 重新创建一个新的 DataLoader 迭代器
      data = next(data_iter, None)    # 再取一个 batch
    inputs, labels = data['inputs'], data['labels']
    logits = model(inputs)
    loss.append(F.cross_entropy(logits, labels).item())
  return torch.tensor(loss).mean().item()


def estimate_loss(model):
  re = {}
  # 将模型切换到评估模式
  model.eval()
  re['train'] = _loss(model, train_loader)
  re['test'] = _loss(model, test_loader)
  # 将模型切换至训练模式
  model.train()
  return re



In [ ]:
estimate_loss(model)

In [ ]:
def train_mlp(model, optimizer, data_loader, epochs=10):
  # 记录模型在训练集上的模型损失
  lossi = []
  for epoch in range(epochs):
    for i, data in enumerate(data_loader, 0):
      inputs, labels = data['inputs'], data['labels']
      optimizer.zero_grad()
      logits = model(inputs)
      loss = F.cross_entropy(logits, labels)
      lossi.append(loss.item())
      loss.backward()
      optimizer.step()
    # 评估模型，并输出结果
    stats = estimate_loss(model)
    train_loss = f'train loss {stats["train"]:.4f}'
    test_loss = f'test loss {stats["test"]:.4f}'
    print(f'epoch {epoch:>2}: {train_loss}, {test_loss}')
  return lossi


In [ ]:
l = train_mlp(model, optim.Adam(model.parameters(), lr=learning_rate), train_loader)

In [ ]:
plt.plot(torch.tensor(l).view(-1, 10).mean(1).numpy())

In [ ]:
# 使用模型来生成文本
context = torch.zeros((1, 10), dtype=torch.long, device=device)
print(''.join(tok.decode(generate(model, context))))